In [4]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

import sys
sys.path.append("..")

import src.support.data_upload_support as dus
import src.support.data_extraction_support as des


In [5]:
enlaces_navegacion = ["https://minimalismbrand.com/collections/hogar",
                      "https://minimalismbrand.com/collections/mochilas",
                      "https://minimalismbrand.com/collections/camisetas-algodon-organico-nino-a",
                      "https://minimalismbrand.com/collections/ropa-minimalism-mujer",
                      "https://minimalismbrand.com/collections/ropa-minimalism"
                      ]

Diccionarios funciones extraccion

In [6]:
diccionario_extract = {
    "nombre": lambda card: re.sub(r"\s+"," ",card.find("h3",{"class":"product-block__title"}).text.strip()),
    "precio": lambda card: card.find("div",{"class":"product-price"}).findChildren("span",recursive=False)[0].text.strip(),
    "precio_original": lambda card: card.find("div",{"class":"product-price"}).findChildren("span",recursive=False)[1].text.strip(),
    "url": lambda card: "https://minimalismbrand.com"+ card.find("a",{"class":"product-link"})["href"]
}


In [7]:
diccionario = {
        "nombre": [],
        "precio": [],
        "precio_original": [],
        "url": []
}
for enlace in enlaces_navegacion:
    print("Enlace principal", enlace)

    # 
    response = requests.get(enlace)
    
    soup = BeautifulSoup(response.text, 'html.parser')

    # enlaces en subcategoría
    enlace_subcategoria = soup.find("a",{"class":"image-overlay__image-link"})

    # SI ES PAGINA SUBCATEGORIAS
    if enlace_subcategoria:

        # ENCONTRAR ENLACES SUBCATEGORIAS, ENTRAR EN CADA PAGINA PRODUCTOS
        lista_enlace_pag_productos = ["https://minimalismbrand.com"+elemento["href"] for elemento in soup.find_all("a",{"class":"image-overlay__image-link"})]

        try:
            lista_nombres_productos = [elemento.find("span").text.strip() for elemento in soup.find_all("a",{"class":"image-overlay__image-link"}) if elemento and elemento.find("span")]
        except:
            # print("except")
            lista_nombres_productos = [elemento.find("span") for elemento in soup.find_all("a",{"class":"image-overlay__image-link"}) if elemento]

            
        # print(lista_nombres_productos)

        for enlace_pag_producto in lista_enlace_pag_productos: 
            # ENCONTRAR EN CADA URL PRODUCTO
            # print("Enlace_pag_producto", enlace_pag_producto)
            response = requests.get(enlace_pag_producto)
            subsoup = BeautifulSoup(response.text, 'html.parser')
            
            diccionario = des.sacar_productos_pagina(subsoup,diccionario_extract, diccionario)

    # SI ES PAGINA ENLACES A PRODUCTOS 
    else:
        diccionario = des.sacar_productos_pagina(soup,diccionario_extract, diccionario)

precios_minimalism = pd.DataFrame(diccionario)
precios_minimalism["precio"] = np.where(precios_minimalism["precio"]=="De", precios_minimalism["precio_original"], precios_minimalism["precio"])
precios_minimalism[["precio","precio_original"]] = precios_minimalism[["precio","precio_original"]].fillna("-")
precios_minimalism[["precio","precio_original"]] = precios_minimalism[["precio","precio_original"]].map(lambda x: x.replace("€",""))
precios_minimalism.shape

Enlace principal https://minimalismbrand.com/collections/hogar
Enlace principal https://minimalismbrand.com/collections/mochilas
Enlace principal https://minimalismbrand.com/collections/camisetas-algodon-organico-nino-a
Enlace principal https://minimalismbrand.com/collections/ropa-minimalism-mujer
Enlace principal https://minimalismbrand.com/collections/ropa-minimalism


(258, 4)

## Cleaning

In [8]:
precios_minimalism = pd.DataFrame(diccionario)
precios_minimalism["precio"] = np.where(precios_minimalism["precio"]=="De", precios_minimalism["precio_original"], precios_minimalism["precio"])
precios_minimalism[["precio","precio_original"]] = precios_minimalism[["precio","precio_original"]].fillna("-")
precios_minimalism[["precio","precio_original"]] = precios_minimalism[["precio","precio_original"]].map(lambda x: x.replace("€",""))
precios_minimalism.shape

(258, 4)

In [9]:
precios_minimalism["url2"] = precios_minimalism["url"].str.split("?",expand=True)[0]
precios_minimalism.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nombre           258 non-null    object
 1   precio           258 non-null    object
 2   precio_original  258 non-null    object
 3   url              258 non-null    object
 4   url2             258 non-null    object
dtypes: object(5)
memory usage: 10.2+ KB


In [10]:
precios_minimalism.drop_duplicates(subset="url2",inplace=True)
precios_minimalism.shape

(80, 5)

In [11]:
precios_minimalism

,nombre,precio,precio_original,url,url2
0,Juego de sábanas · Algodón orgánico - 105x200,70.00,-,https://minimalismbrand.com/products/juego-de-...,https://minimalismbrand.com/products/juego-de-...
4,Toalla lavabo · Algodón orgánico - Blanco,9.00,-,https://minimalismbrand.com/products/toalla-la...,https://minimalismbrand.com/products/toalla-la...
6,Toalla de ducha · Algodón orgánico - Blanco,17.00,-,https://minimalismbrand.com/products/toalla-du...,https://minimalismbrand.com/products/toalla-du...
8,Toallas baño pack mediano · Algodón orgánico,24.00,26.00,https://minimalismbrand.com/products/toallas-b...,https://minimalismbrand.com/products/toallas-b...
9,Gel sólido uso diario x3,20.00,22.50,https://minimalismbrand.com/products/gel-solid...,https://minimalismbrand.com/products/gel-solid...
...,...,...,...,...,...
180,Calzoncillos Slips Orgánicos X3,32.00,36.00,https://minimalismbrand.com/products/calzoncil...,https://minimalismbrand.com/products/calzoncil...
181,Calzoncillos Slips Orgánicos X5,50.00,60.00,https://minimalismbrand.com/products/calzoncil...,https://minimalismbrand.com/products/calzoncil...
182,Calzoncillos Slips Orgánicos X7,68.00,84.00,https://minimalismbrand.com/products/calzoncil...,https://minimalismbrand.com/products/calzoncil...
210,Pantalón orgánico Slim Hombre,70.00,-,https://minimalismbrand.com/products/pantalon-...,https://minimalismbrand.com/products/pantalon-...


Añadir informaciones de score etc

In [12]:
precios_minimalism[["score", "reviews", "vertical", "description", "politica_envios", 
                          "impacto", "desglose_costes", "material2", "cultivo", "confeccion", "logistica"]] = precios_minimalism["url2"].apply(lambda x: des.extraer_info_producto(x)).to_list()

In [13]:
precios_minimalism

,nombre,precio,precio_original,url,url2,score,reviews,vertical,description,politica_envios,impacto,desglose_costes,material2,cultivo,confeccion,logistica
0,Juego de sábanas · Algodón orgánico - 105x200,70.00,-,https://minimalismbrand.com/products/juego-de-...,https://minimalismbrand.com/products/juego-de-...,3.7,3,Esenciales para tu hogar,--Descripción-- -Ropa de cama suave al tacto c...,---Política de envíos--Envío a España y Portug...,NaN,---Desglose de costes--El precio final de este...,NaN,1. CULTIVO DEL ALGODÓN # Procedencia del algod...,2. CONFECCIÓN Y MANUFACTURA # Algodón certific...,3. CENTRO LOGÍSTICO Y MANUFACTURA # Almacén / ...
4,Toalla lavabo · Algodón orgánico - Blanco,9.00,-,https://minimalismbrand.com/products/toalla-la...,https://minimalismbrand.com/products/toalla-la...,4.2,9,Esenciales para tu baño,--Descripción---Toallas de rizo absorbente y s...,---Política de envíos--Envío a España y Portug...,NaN,---Desglose de costes--El precio final de este...,NaN,1. CULTIVO DEL ALGODÓN # Procedencia del algod...,2. CONFECCIÓN Y MANUFACTURA # Algodón certific...,3. CENTRO LOGÍSTICO Y MANUFACTURA # Almacén / ...
6,Toalla de ducha · Algodón orgánico - Blanco,17.00,-,https://minimalismbrand.com/products/toalla-du...,https://minimalismbrand.com/products/toalla-du...,4.2,9,Esenciales para tu baño,--Descripción--Toallas de rizo absorbente y su...,---Política de envíos--Envío a España y Portug...,NaN,---Desglose de costes--El precio final de este...,NaN,1. CULTIVO DEL ALGODÓN # Procedencia del algod...,2. CONFECCIÓN Y MANUFACTURA # Algodón certific...,3. CENTRO LOGÍSTICO Y MANUFACTURA # Almacén / ...
8,Toallas baño pack mediano · Algodón orgánico,24.00,26.00,https://minimalismbrand.com/products/toallas-b...,https://minimalismbrand.com/products/toallas-b...,4.2,9,Esenciales para tu baño,--Descripción---Toallas de rizo absorbente y s...,---Política de envíos--Envío a España y Portug...,NaN,---Desglose de costes--El precio final de este...,NaN,1. CULTIVO DEL ALGODÓN # Procedencia del algod...,2. CONFECCIÓN Y MANUFACTURA # Algodón certific...,3. CENTRO LOGÍSITCO Y MANUFACTURA # Almacén / ...
9,Gel sólido uso diario x3,20.00,22.50,https://minimalismbrand.com/products/gel-solid...,https://minimalismbrand.com/products/gel-solid...,NaN,NaN,Esenciales para tu baño,--Descripción--Gel sólido de 80 g para uso dia...,---Política de envíos--Envío a España y Portug...,NaN,---Desglose de costes--El precio final de este...,NaN,1. CULTIVO DEL ALGODÓN # Procedencia del algod...,2. CONFECCIÓN Y MANUFACTURA # Algodón certific...,3. CENTRO LOGÍSITCO Y MANUFACTURA # Almacén / ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,Calzoncillos Slips Orgánicos X3,32.00,36.00,https://minimalismbrand.com/products/calzoncil...,https://minimalismbrand.com/products/calzoncil...,NaN,NaN,Calzoncillos,--Descripción---Calzoncillos estilo slip fabri...,---Política de envíos--Envío a España y Portug...,---Impacto ambiental-- -¿Por qué decimos que n...,---Desglose de costes--El precio final de este...,NaN,1. CULTIVO DEL ALGODÓN # Procedencia del algod...,2. CONFECCIÓN Y MANUFACTURA # Algodón certific...,3. CENTRO LOGÍSITCO Y MANUFACTURA # Almacén / ...
181,Calzoncillos Slips Orgánicos X5,50.00,60.00,https://minimalismbrand.com/products/calzoncil...,https://minimalismbrand.com/products/calzoncil...,NaN,NaN,Calzoncillos,--Descripción---Calzoncillos estilo slip fabri...,---Política de envíos--Envío a España y Portug...,---Impacto ambiental-- -¿Por qué decimos que n...,---Desglose de costes--El precio final de este...,NaN,1. CULTIVO DEL ALGODÓN # Procedencia del algod...,2. CONFECCIÓN Y MANUFACTURA # Algodón certific...,3. CENTRO LOGÍSITCO Y MANUFACTURA # Almacén / ...
182,Calzoncillos Slips Orgánicos X7,68.00,84.00,https://minimalismbrand.com/products/calzoncil...,https://minimalismbrand.com/products/calzoncil...,NaN,NaN,Calzoncillos,--Descripción---Calzoncillos estilo slip fabri...,---Política de envíos--Envío a España y Portug...,---Impacto 

In [15]:
precios_minimalism.to_csv("../datos/precios_minimalism.csv")

In [2]:
dus.upload_to_drive("../datos/precios_minimalism.csv", '1vfttgGDHa3-eCMEN79QAue6aI3Gc1vvp')

Archivo subido con ID: 1kD14LR-klyEbG_bGpZJCwOP1gpNzg-ub


## Pruebas beautiful soup pagina producto

In [15]:
url_pag_producto = "https://minimalismbrand.com/products/camiseta-algodon-organico?variant=34948534108317"
response = requests.get(url_pag_producto)

soup = BeautifulSoup(response.text, 'html.parser')

Conseguir vertical

In [16]:
" ".join(soup.find("div",{"class":"container pango-breadcrumb"}).text.strip().split()[2:])

'Camisetas algodón orgánico'

In [ ]:
precios_minimalism[["score","reviews","vertical","description","politica_envios","impacto","desglose_costes", "material2","cultivo","confeccion","logistica"]] = precios_minimalism["url2"].apply(lambda x: extraer_info_producto(x)).to_list()

Conseguir score

In [ ]:
score = soup.find("span",{"class":"stamped-badge-caption"})["data-rating"]


Conseguir reviews

In [ ]:
n_reviews = soup.find("span",{"class":"stamped-badge-caption"})["data-reviews"]